In [1]:
# Import necessary packages
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
# Site URL
url='https://alsod.ac.uk/'#Create a handle, page, to handle the contents of the website

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse HTML code for the entire site
soup = BeautifulSoup(html_content, "lxml")
#print(soup.prettify()) # print the parsed data of html

In [5]:
all_genes = soup.find_all("table", attrs={"class": "table"})
table1 = all_genes[1]
body = table1.find_all("tr")
# Head values (Column names) are the first items of the body list
head = body[0] # 0th item is the header row
body_rows = body[1:] # All other items becomes the rest of the rows

# Lets now iterate through the head HTML code and make list of clean headings

# Declare empty list to keep Columns names
headings = []
for item in head.find_all("th"): # loop through all th elements
    # convert the th elements to text and strip "\n"
    item = (item.text).rstrip("\n")
    # append the clean column name to headings
    headings.append(item)
# Next is now to loop though the rest of the rows

#print(body_rows[0])
all_rows = [] # will be a list for list for all rows
for row_num in range(len(body_rows)): # A row at a time
    row = [] # this will old entries for one row
    for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
        # row_item.text removes the tags from the entries
        # the following regex is to remove \xa0 and \n and comma from row_item.text
        # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
        aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
        #append aa to row - note one row entry is being appended
        row.append(aa)
    # append one row to all_rows
    all_rows.append(row)
    
# We can now use the data on all_rowsa and headings to make a table
# all_rows becomes our data and headings the column names
df = pd.DataFrame(data=all_rows,columns=headings)
df.head()

,,Gene symbol,Gene name,Category
0,,AGT,angiotensinogen,Tenuous
1,,ALAD,aminolevulinate dehydratase,Tenuous
2,,ALS2,alsin Rho guanine nucleotide exchange factor ALS2,Tenuous
3,,ALS3,amyotrophic lateral sclerosis 3 (autosomal dom...,Tenuous
4,,ALS7,amyotrophic lateral sclerosis 7,Tenuous


In [31]:
def get_gene(gene):
    url2 = "https://alsod.ac.uk/output/gene.php/"
    url2 = url2 + gene
    # Make a GET request to fetch the raw HTML content
    html_content = requests.get(url2).text

    # Parse HTML code for the entire site
    soup = BeautifulSoup(html_content, "lxml")
    this_gene = soup.find_all("table", attrs={"class": "table"})
    table2 = this_gene[1]
    body = table2.find_all("tr")
    # Head values (Column names) are the first items of the body list
    head = body[0] # 0th item is the header row
    body_rows = body[1:] # All other items becomes the rest of the rows

    # Lets now iterate through the head HTML code and make list of clean headings

    # Declare empty list to keep Columns names
    headings = []
    for item in head.find_all("th"): # loop through all th elements
        # convert the th elements to text and strip "\n"
        item = (item.text).rstrip("\n")
        # append the clean column name to headings
        headings.append(item)
    # Next is now to loop though the rest of the rows

    #print(body_rows[0])
    all_rows = [] # will be a list for list for all rows
    for row_num in range(len(body_rows)): # A row at a time
        row = [] # this will old entries for one row
        for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
            # row_item.text removes the tags from the entries
            # the following regex is to remove \xa0 and \n and comma from row_item.text
            # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
            aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
            #append aa to row - note one row entry is being appended
            row.append(aa)
        # append one row to all_rows
        all_rows.append(row)

    # We can now use the data on all_rowsa and headings to make a table
    # all_rows becomes our data and headings the column names
    df2 = pd.DataFrame(data=all_rows,columns=headings)
    df2['gene']= gene
    return df2

In [32]:
genes_to_check = df[df['Category'].isin(['Moderate evidence','Definitive ALS gene','Strong evidence'])]['Gene symbol'].tolist()

In [38]:
appended_data = pd.DataFrame()
for g in genes_to_check:
    try:
        data = get_gene(g)
        print(data)
        appended_data=data_you_need.append(data,ignore_index=True)
    except:
        print(f'{g} doesnt have variants')
        
appended_data

    Mnemonic Chromosome Position hg19                   HGVS # publications  \
0       A24T         14      21161793      NM_001145:c.70G>A              1   
1       C39W         14      21161912   NM_001145.4:c.189C>G              2   
2      C446T                                                              1   
3       D22G         14      21161860   NM_001145.4:c.137A>G              1   
4      F-13L         14      21161759    NM_001145.4:c.36C>T              2   
5      F-13S         14      21161758    NM_001145.4:c.35T>C              2   
6      F100I                            NM_001145.4:c.298T>A              2   
7      G-10D                                                              2   
8       G20G         14      21161855   NM_001145.4:c.132C>G              1   
9      H114R         14      21162136   NM_001145.4:c.413A>G              2   
10      H84R         14      21162046   NM_001145.4:c.323A>G              1   
11      I46V         14      21161931   NM_001145.4:

  Mnemonic Chromosome Position hg19                   HGVS # publications  \
0    E954V          3       1444045  NM_014461.2:c.2861A>T              1   
1    F314V          3       1363512   NM_014461.2:c.940T>G              1   

  # patients   gene  
0          0  CNTN6  
1          0  CNTN6  
  Mnemonic Chromosome Position hg19                  HGVS # publications  \
0    R169C         16      21279043  NM_001888.3:c.505C>T              1   

  # patients  gene  
0          1  CRYM  
CSNK1G3 doesnt have variants
  Mnemonic Chromosome Position hg19 HGVS # publications # patients    gene
0    T280M                                            1          0  CX3CR1
1    V249I                                            1          0  CX3CR1
  Mnemonic Chromosome Position hg19                  HGVS # publications  \
0    R199W         12     109288126  NM_001917.4:c.595C>T              1   
1     R38H         12     109278895  NM_001917.4:c.113G>A              1   

  # patients gene  
0   

  Mnemonic Chromosome Position hg19 HGVS # publications # patients   gene
0     A86G                                            1          0  NIPA1
1    I120M                                            1          0  NIPA1
2     I81T                                            1          0  NIPA1
3    M189I                                            1          0  NIPA1
4    P221L                                            1          0  NIPA1
5    R281Q                                            1          0  NIPA1
6    V162M                                            1          0  NIPA1
          Mnemonic Chromosome  Position hg19                         HGVS  \
0   1242+1G>A_insA                                                          
1     691_692insAG         10  13160952 - 10   NM_021980.4:c.691_692insAG   
2            A134A                                                          
3            A136V         10       13154490         NM_00108211:c.407C>T   
4            A481V     

SCFD1 doesnt have variants
SMN1 doesnt have variants
    Mnemonic Chromosome Position hg19                  HGVS # publications  \
0      A140A         21      33040849  NM_000454.4:c.423T>A              3   
1      A140G         21      33040848  NM_000454.4:c.422C>G              2   
2      A145G         21      33040863  NM_000454.4:c.437C>G              1   
3      A145T         21      33040862  NM_000454.4:c.436G>A              1   
4      A152P                                                             0   
..       ...        ...           ...                   ...            ...   
212     V87A         21      33039594  NM_000454.4:c.263T>C              1   
213     V87M         21      33039593  NM_000454.4:c.262G>A              1   
214     V94A         21      33039615  NM_000454.4:c.284T>C              1   
215     V97L         21      33039623  NM_000454.4:c.292G>C              1   
216     V97M         21      33039623  NM_000454.4:c.292G>A              2   

    # pati

     Mnemonic Chromosome Position hg19                HGVS # publications  \
0       A383T                                                           1   
1       D438N                                                           1   
2        G43V                                                           1   
3       I234T          2     220115720  NM_006000:c.701T>C              1   
4        I42T          2     220116831  NM_006000:c.125T>C              1   
5   IVS2+4A>G                                                           1   
6       K430N                                                           1   
7       R215C                                                           1   
8       R320C                                                           1   
9       R320H                                                           1   
10      T145P                                                           1   
11      T349S                                                           1   

""


In [37]:
appended_data

""
